In [ ]:
import math
from timeit import default_timer as timer

import bayesfunc as bf
import matplotlib
import matplotlib.pyplot as plt
import torch as t
import torch.nn as nn
from torch.distributions import Categorical, Normal
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.data import DataLoader

from data.regression import RegressionDataset, generate_regression_data
from tqdm.auto import tqdm
import numpy as np

%load_ext autoreload
%autoreload 2

matplotlib.style.use('default')
plt.rcParams.update({'axes.titlesize': 'large', 'axes.labelsize': 'medium'})
colors_hex = {'blue': '#1F77B4', 'orange': '#FF7F0E', 'green': '#2CA02C'}
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

In [ ]:
t.cuda.is_available()
device = t.device("cuda:0" if t.cuda.is_available() else "cpu")
print(device)

t.manual_seed(0)

How to download, init and use a UCI dataset

In [ ]:
from uci import UCI

In [4]:
protein = UCI('protein', 1, 10000)
protein.trainset

Using Ashman's code

In [28]:
from data.preprocess_data import download_datasets, process_dataset, datasets, protein_config

In [27]:
download_datasets(root_dir='./data', datasets={'protein': datasets['protein']})

0/45730 rows had missing data



In [32]:
process_dataset('data/uci/protein', protein_config)

Input  shape: (45730, 9)
Output shape: (45730, 1)
